In [1]:
import json, os
from DeviceObjects import ALL_DEVICE_TYPES
import devices
from typing import Any

def serialize_device_types() -> dict[str, Any]:
    result = {}

    for device_name, device_cls in ALL_DEVICE_TYPES.items():
        skill_data = []
        for skill in device_cls.skills:
            try:
                skill_data.append(skill.dict())
            except Exception as e:
                skill_data.append({
                    "id": getattr(skill, 'id', 'unknown'),
                    "error": str(e)
                })

        result[device_name] = {
            "skills": skill_data
        }

    return result



data = serialize_device_types()
# print(json.dumps(data, indent=2, ensure_ascii=False))
output_path = "exported_device_skills.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"\n✅ JSON 파일 저장 완료: {os.path.abspath(output_path)}")


✅ JSON 파일 저장 완료: d:\desktop\창통설\skills_일부\integration\exported_device_skills.json


In [4]:
import json

with open("exported_device_skills.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    # 추출
    extracted = {}

    for device_name, device_info in data.items():
        skills_summary = []
        for skill in device_info.get("skills", []):
            skills_summary.append({
                "id": skill["id"],
                "descriptor": skill["descriptor"]
            })
        extracted[device_name] = {"skills": skills_summary}

    output_path = "exported_devices.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(extracted, f, indent=2, ensure_ascii=False)

    print(f"\n✅ JSON 파일 저장 완료: {os.path.abspath(output_path)}")


✅ JSON 파일 저장 완료: d:\desktop\창통설\skills_일부\integration\exported_devices.json


In [8]:
import json
import os

with open("exported_device_skills.json", "r", encoding="utf-8") as f:
    data = json.load(f)

    # 중복 제거용 딕셔너리
    unique_skills = {}

    for device_info in data.values():
        for skill in device_info.get("skills", []):
            skill_id = skill["id"]
            if skill_id not in unique_skills:
                unique_skills[skill_id] = skill

    # 결과 포맷
    extracted_skills = list(unique_skills.values())

    # 저장
    output_path = "exported_skills.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(extracted_skills, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 중복 제거된 스킬 JSON 저장 완료: {os.path.abspath(output_path)}")


✅ 중복 제거된 스킬 JSON 저장 완료: d:\desktop\창통설\skills_일부\integration\exported_skills.json


## 압축

In [12]:
import json

with open("exported_device_skills.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    # 추출
    extracted = {}

    for device_name, device_info in data.items():
        skills_summary = {}
        for skill in device_info.get("skills", []):
            skills_summary[skill["id"]] = skill["descriptor"]
        extracted[device_name] = skills_summary

    output_path = "compressed_devices.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(extracted, f, indent=2, ensure_ascii=False)

    print(f"\n✅ JSON 파일 저장 완료: {os.path.abspath(output_path)}")


✅ JSON 파일 저장 완료: d:\desktop\창통설\skills_일부\integration\compressed_devices.json


In [5]:
import json
import os

words = {
    "descriptor": "info", 
    "values": "values", 
    "functions": "functions", 
    "arguments": "arguements",
    "return_type": "return_type",
}

with open("exported_device_skills.json", "r", encoding="utf-8") as f:
    data = json.load(f)

    # 중복 제거용 딕셔너리
    unique_skills = {}

    for device_info in data.values():
        for skill in device_info.get("skills", []):
            skill_id = skill["id"]
            del skill["id"]

            skill["info"] = skill.pop("descriptor")

            original_values = skill.pop("values")
            converted_values = {
                item["id"]: {
                    (words[k] if k in words else k): v
                    for k, v in item.items() if k != "id" and v!=None
                }
                for item in original_values
            }
            if converted_values != {}:
                skill[words["values"]] = converted_values

            original_functions = skill.pop("functions")

            converted_functions = {}
            for func in original_functions:  # func는 dict
                func_id = func["id"]
                converted_item = {
                    (words[k] if k in words else k): v
                    for k, v in func.items() if k != "id" and v!=None
                }

                # args 내부도 처리
                if "args" in converted_item:
                    converted_item["args"] = {
                        arg["id"]:{
                            (words[k] if k in words else k): v
                            for k, v in arg.items() if k!="id" and v!=None
                        }
                        for arg in converted_item["args"]
                    }

                converted_functions[func_id] = converted_item

            if converted_functions != {}:
                skill[words["functions"]] = converted_functions

            original_enum = skill.pop("enum", None)
            converted_enum = {}

            if original_enum:
                enum_id = original_enum["id"]
                descriptor = original_enum.get("descriptor", "")
                enums_list = original_enum.get("enums", [])

                # descriptor 파싱: "• key - 설명" 형식
                parsed_items = {}
                for line in descriptor.split("\n"):
                    line = line.strip("•").strip()
                    if " - " in line:
                        key, desc = line.split(" - ", 1)
                        parsed_items[key.strip()] = desc.strip()

                converted_enum[enum_id] = parsed_items
        
            if converted_enum != {}:
                skill["enum"] = converted_enum

            if skill_id not in unique_skills:
                unique_skills[skill_id] = skill

    # 결과 포맷
    extracted_skills = unique_skills

    # 저장
    output_path = "compressed_skills.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(extracted_skills, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 중복 제거된 스킬 JSON 저장 완료: {os.path.abspath(output_path)}")


✅ 중복 제거된 스킬 JSON 저장 완료: d:\desktop\창통설\skills_일부\integration\compressed_skills.json


## 재병합

In [7]:
import json 

def merge_device_skill(devices: dict, skills: dict) -> dict:
    merged = {}
    for device_name, skill_map in devices.items():
        device_skills = []
        for skill_id, skill_desc in skill_map.items():
            skill_info = skills.get(skill_id, {})
            device_skills.append({skill_id:{
                "info": skill_info.get("info", skill_map[skill_id]),
                "values": skill_info.get("values", {}),
                "functions": skill_info.get("functions", {}),
                "enum": skill_info.get("enum", {}),
            }})
        merged[device_name] = {"skills": device_skills}
    return merged

with open("compressed_devices.json", "r", encoding="utf-8") as f1:
    with open("compressed_skills.json", "r", encoding="utf-8") as f2:
        devices = json.load(f1)
        skills = json.load(f2)
        merged_data = merge_device_skill(devices, skills)
        output_path = "compressed_devices_skills.json"
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(merged_data, f, indent=2, ensure_ascii=False)
